In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark = SparkSession.builder.getOrCreate()

In [2]:
path_dir = './data_source/data_2014/2014'
bb_2014 = spark.read.csv(path_dir, header=True, inferSchema =True)

In [12]:
bb_2014.select(bb_2014.columns[:8]).show(5)

+----------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+
|      date| serial_number|               model|capacity_bytes|failure|smart_1_normalized|smart_1_raw|smart_2_normalized|
+----------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+
|2014-12-16|MJ0351YNG9Z0XA|Hitachi HDS5C3030...| 3000592982016|      0|               100|          0|               135|
|2014-12-16|MJ0351YNG9WJSA|Hitachi HDS5C3030...| 3000592982016|      0|               100|          0|               136|
|2014-12-16|MJ0351YNG9Z7LA|Hitachi HDS5C3030...| 3000592982016|      0|               100|          0|               136|
|2014-12-16|MJ0351YNGABYAA|Hitachi HDS5C3030...| 3000592982016|      0|               100|          0|               136|
|2014-12-16|PL2331LAGN2YTJ|HGST HMS5C4040BLE640| 4000787030016|      0|               100|          0|               134|
+----------+------------

In [13]:
bb_2014.createOrReplaceTempView("bb_2014")

In [14]:
spark.sql("select serial_number from bb_2014 where failure=1").show(5)

+---------------+
|  serial_number|
+---------------+
|WD-WCAV5V395871|
|WD-WCC4N0919016|
| PL1310LAG0NBWA|
|       6XW00HYZ|
|       S1F0B4PP|
+---------------+
only showing top 5 rows



In [15]:
bb_2014.where("failure=1").select(F.col('serial_number')).show(5)

+---------------+
|  serial_number|
+---------------+
|WD-WCAV5V395871|
|WD-WCC4N0919016|
| PL1310LAG0NBWA|
|       6XW00HYZ|
|       S1F0B4PP|
+---------------+
only showing top 5 rows



In [16]:
spark.sql("""select model, min(capacity_bytes/pow(1024,3)) min_GB,
                           max(capacity_bytes/pow(1024,3)) max_GB
                           from bb_2014 group by 1 order by 3 desc""").show(5)

+--------------------+------------------+-------------------+
|               model|            min_GB|             max_GB|
+--------------------+------------------+-------------------+
|         ST4000DM000| 3726.023277282715| 5.59103270818843E8|
|HGST HMS5C4040ALE640|1048.0001196861267|9.997684600476837E7|
|        WDC WD60EFRX|  5589.02986907959|   5589.02986907959|
|         ST6000DX000|  5589.02986907959|   5589.02986907959|
|        WDC WD40EFRX| 3726.023277282715|  3726.023277282715|
+--------------------+------------------+-------------------+
only showing top 5 rows



In [17]:
bb_2014.groupby(F.col('model')).agg(F.min(F.col('capacity_bytes')/F.pow(F.lit(1024),3)).alias('min_GB'),
                                   F.max(F.col('capacity_bytes')/F.pow(F.lit(1024),3)).alias('max_GB')).orderBy(F.col('max_GB'), ascending=False).show(5)

+--------------------+------------------+-------------------+
|               model|            min_GB|             max_GB|
+--------------------+------------------+-------------------+
|         ST4000DM000| 3726.023277282715| 5.59103270818843E8|
|HGST HMS5C4040ALE640|1048.0001196861267|9.997684600476837E7|
|        WDC WD60EFRX|  5589.02986907959|   5589.02986907959|
|         ST6000DX000|  5589.02986907959|   5589.02986907959|
|        WDC WD40EFRX| 3726.023277282715|  3726.023277282715|
+--------------------+------------------+-------------------+
only showing top 5 rows

